In [1]:
#!pip install vectorbt

In [2]:
#url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
#ext = '0.4.0-oneiric1_amd64.deb -qO'
#!wget $url/libta-lib0_$ext libta.deb
#!wget $url/ta-lib0-dev_$ext ta.deb
#!dpkg -i libta.deb ta.deb
#!pip install ta-lib
import talib

In [3]:
import pandas as pd
from datetime import datetime
import vectorbt as vbt
import pandas as pd
import numpy as np

In [4]:
market_data = pd.read_csv('./WIN$N_M1.csv', sep= '	')

In [5]:
len(market_data)

282763

In [6]:
#market_data = market_data.iloc[200000:,:]

Gerando Features a partir da coluna DATE e TIME

In [7]:
market_data['<HOUR>'] = market_data['<TIME>'].apply(lambda x : int(datetime.strptime(x, '%H:%M:%S').hour))
market_data['<MINUTE>'] = market_data['<TIME>'].apply(lambda x : int(datetime.strptime(x, '%H:%M:%S').minute))
market_data['<DAY>'] = market_data['<DATE>'].apply(lambda x : int(datetime.strptime(x, "%Y.%m.%d").day))
market_data['<YEAR>'] = market_data['<DATE>'].apply(lambda x : int(datetime.strptime(x, "%Y.%m.%d").year))
market_data['<WEEKDAY>'] = market_data['<DATE>'].apply(lambda x : int(datetime.strptime(x, "%Y.%m.%d").weekday()))
market_data['<MONTH>'] = market_data['<DATE>'].apply(lambda x : int(datetime.strptime(x, "%Y.%m.%d").month))
#Flag é final de semana, porém mini indice não tem pregão aos finais de semana...
#market_data['<WEEKEND>'] = market_data['<WEEKDAY>'].apply(lambda x : x >= 5).astype(int)


Indicadores de análise técnica

In [8]:
rsi_window = 14
rsi_sell_value = 70
rsi_buy_value = 30
sma_window = 30
roc_window = 10
ema_window = 9
wma_window = 30
macd_fastperiod=12
macd_slowperiod=26
macd_signalperiod=9

In [9]:
market_data['Close'] = market_data['<CLOSE>'].apply(lambda x : float(x))

In [10]:
#Insert RSI Info to the dataset:
# RSI Value -- Rsi Signal to buy -- RSI Signal to sell

rsi = vbt.RSI.run(market_data['Close'], window = rsi_window).rsi
            #np.where(condition, value to apply if true, value to apply otherwise)
rsi_sell = np.where(rsi > rsi_sell_value, -1,  0)
rsi_buy = np.where(rsi < rsi_buy_value, 1,  0)  
market_data['RSI'] = rsi
market_data['RSI_Buy'] = rsi_buy
market_data['RSI_Sell'] = rsi_sell

#Insert SMA Info to the dataset:
sma = vbt.MA.run(market_data['Close'], window = sma_window).ma
sma_sell = np.where(sma > market_data['Close'], -1,  0)
sma_buy = np.where(sma < market_data['Close'], 1,  0)  
market_data['SMA'] = sma
market_data['SMA_Buy'] = sma_buy
market_data['SMA_Sell'] = sma_sell

#Insert ROC Info to the dataset:

ROC = vbt.IndicatorFactory.from_talib('ROC')

roc = ROC.run(market_data['Close'], timeperiod = roc_window).real
roc_buy = np.where(roc > 0, 1, 0)
roc_sell = np.where(roc < 0, -1, 0)
market_data['ROC'] = roc
market_data['ROC_Buy'] = roc_buy
market_data['ROC_Sell'] = roc_sell

#Insert EMA Info to the dataset:

EMA = vbt.IndicatorFactory.from_talib('EMA')

ema = EMA.run(market_data['Close'], timeperiod = ema_window).real
ema_sell = np.where(ema > market_data['Close'], -1,  0)
ema_buy = np.where(ema < market_data['Close'], 1,  0)
market_data['EMA'] = ema
market_data['EMA_Buy'] = ema_buy
market_data['EMA_Sell'] = ema_sell  

#Insert WMA Info to the dataset:

WMA = vbt.IndicatorFactory.from_talib('WMA')

wma = WMA.run(market_data['Close'], timeperiod = wma_window).real
wma_sell = np.where(wma > market_data['Close'], -1,  0)
wma_buy = np.where(wma < market_data['Close'], 1,  0)
market_data['WMA'] = wma
market_data['WMA_Buy'] = wma_buy
market_data['WMA_Sell'] = wma_sell  


#Insert MACD Info to the dataset:

MACD = vbt.IndicatorFactory.from_talib('MACD')

macd = MACD.run(market_data['Close'], fastperiod = macd_fastperiod, slowperiod = macd_slowperiod, signalperiod = macd_signalperiod).macd
macd_buy = np.where(macd > 0, 1,  0)
macd_sell = np.where(macd < 0, -1,  0)
market_data['MACD'] = macd
market_data['MACD_Buy'] = macd_buy
market_data['MACD_Sell'] = macd_sell  


#The target value is a long Weight Moving Average WMA, 100 periods
#In order to classify the trend in:
# 1 --> uptrend -- bullish
# 0 --> sideways -- neutral
#-1 --> downtrend -- bearish

print(f"Length of the columns of the dataset: {len(market_data.columns)}")
print(market_data.columns)
market_data.head()

Length of the columns of the dataset: 34
Index(['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>',
       '<VOL>', '<SPREAD>', '<HOUR>', '<MINUTE>', '<DAY>', '<YEAR>',
       '<WEEKDAY>', '<MONTH>', 'Close', 'RSI', 'RSI_Buy', 'RSI_Sell', 'SMA',
       'SMA_Buy', 'SMA_Sell', 'ROC', 'ROC_Buy', 'ROC_Sell', 'EMA', 'EMA_Buy',
       'EMA_Sell', 'WMA', 'WMA_Buy', 'WMA_Sell', 'MACD', 'MACD_Buy',
       'MACD_Sell'],
      dtype='object')


,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,<HOUR>,...,ROC_Sell,EMA,EMA_Buy,EMA_Sell,WMA,WMA_Buy,WMA_Sell,MACD,MACD_Buy,MACD_Sell
0,2020.10.15,11:49:00,98565,98575,98530,98575,2373,9001,5,11,...,0,NaN,0,0,NaN,0,0,NaN,0,0
1,2020.10.15,11:50:00,98570,98625,98570,98600,5483,19111,5,11,...,0,NaN,0,0,NaN,0,0,NaN,0,0
2,2020.10.15,11:51:00,98595,98600,98535,98580,4572,17663,5,11,...,0,NaN,0,0,NaN,0,0,NaN,0,0
3,2020.10.15,11:52:00,98575,98620,98535,98615,3536,13143,5,11,...,0,NaN,0,0,NaN,0,0,NaN,0,0
4,2020.10.15,11:53:00,98620,98625,98575,98610,3729,13702,5,11,...,0,NaN,0,0,NaN,0,0,NaN,0,0


Gerando colunas, a partir de polinômios de outras colunas

In [11]:
from sklearn import preprocessing

def generate_polynomial(dataset, columns, degree=2):
    # initialize polynomial features class object
    # for two-degree polynomial features
    pf = preprocessing.PolynomialFeatures(
    degree=degree,
    interaction_only=False,
    include_bias=False
    )

    pf.fit(dataset[columns])
    poly_feats = pf.transform(dataset[columns])
    num_feats = poly_feats.shape[1]

    df_transformed = pd.DataFrame(
    poly_feats[:,2:],
            # A X B                                     A2                         B2                     
    columns=[str(columns[0])+'X'+str(columns[1]), str(columns[0]) + "2", str(columns[1]) + "2"]
    )
    return df_transformed, pf

In [12]:
market_data = market_data.dropna()
market_data = market_data.reset_index(drop=True)

In [13]:
#Polinomial features
# “a”, “b”, “ab”, “a2” and “b2”
#close X vol
close_vol_df, close_vol_pf = generate_polynomial(market_data, ['<CLOSE>', '<VOL>'])
market_data = pd.concat([market_data, close_vol_df], axis=1)
#close X RSI
close_rsi_df, close_rsi_pf = generate_polynomial(market_data, ['<CLOSE>', 'RSI'])
market_data = pd.concat([market_data, close_rsi_df], axis=1)
#close X SMA
close_sma_df, close_sma_pf = generate_polynomial(market_data, ['<CLOSE>', 'SMA'])
market_data = pd.concat([market_data, close_sma_df], axis=1)
#close X ROC
close_roc_df, close_roc_pf = generate_polynomial(market_data, ['<CLOSE>', 'ROC'])
market_data = pd.concat([market_data, close_roc_df], axis=1)
#close X EMA
close_ema_df, close_ema_pf = generate_polynomial(market_data, ['<CLOSE>', 'EMA'])
market_data = pd.concat([market_data, close_ema_df], axis=1)
#close X WMA
close_wma_df, close_wma_pf = generate_polynomial(market_data, ['<CLOSE>', 'WMA'])
market_data = pd.concat([market_data, close_wma_df], axis=1)
#close X MACD
close_macd_df, close_macd_pf = generate_polynomial(market_data, ['<CLOSE>', 'MACD'])
market_data = pd.concat([market_data, close_macd_df], axis=1)


Gerando combinação de colunas

In [14]:
from itertools import combinations
def combinate_columns(df, categorical_columns):
    combi = list(combinations(categorical_columns, 2))
    for c1, c2 in combi:
        df.loc[
            :,
            c1 + "_" + c2
        ] = df[c1].astype(str) + "_" + df[c2].astype(str)
    return df

In [15]:
#Combinatorial features
# COMB (BUY SIGNALS) (2)
market_data = combinate_columns(market_data, ['RSI_Buy', 'SMA_Buy', 'ROC_Buy', 'EMA_Buy', 'WMA_Buy', 'MACD_Buy'])
# COMB (SELL SIGNALS) (2)
market_data = combinate_columns(market_data, ['RSI_Sell', 'SMA_Sell', 'ROC_Sell', 'EMA_Sell', 'WMA_Sell', 'MACD_Sell'])

Transformando colunas relativas a tempo, em seno e cosseno

In [16]:
def cyclic_transform(column):
    max_val = column.max()
    sine = [np.sin((2 * np.pi * i ) / max_val) for i in list(column)]
    cosine = [np.cos((2 * np.pi * i ) / max_val) for i in list(column)]
    return sine, cosine

In [17]:
#Seasonal Features - SIN COS
# '<HOUR>', 
#market_data['<HOUR_SIN>'], market_data['<HOUR_COSINE>'] = cyclic_transform(market_data['<HOUR>'])
# '<MINUTE>', 
#market_data['<MINUTE_SIN>'], market_data['<MINUTE_COSINE>'] = cyclic_transform(market_data['<MINUTE>'])
# '<DAY>', 
#market_data['<DAY_SIN>'], market_data['<DAY_COSINE>'] = cyclic_transform(market_data['<DAY>'])
# '<YEAR>',
#market_data['<YEAR_SIN>'], market_data['<YEAR_COSINE>'] = cyclic_transform(market_data['<YEAR>'])
# '<WEEKDAY>', 
#market_data['<WEEKDAY_SIN>'], market_data['<WEEKDAY_COSINE>'] = cyclic_transform(market_data['<WEEKDAY>'])
# '<MONTH>',
#market_data['<MONTH_SIN>'], market_data['<MONTH_COSINE>'] = cyclic_transform(market_data['<MONTH>'])

In [18]:
def generate_stats_columns(dataframe, column, window):
    dataframe[f'Max{column}_{window}'] = dataframe[column].rolling(window).max()
    dataframe[f'Mean{column}_{window}'] = dataframe[column].rolling(window).mean()
    dataframe[f'Min{column}_{window}'] = dataframe[column].rolling(window).min()
    dataframe[f'Std{column}_{window}'] = dataframe[column].rolling(window).std()
    dataframe[f'Var{column}_{window}'] = dataframe[column].rolling(window).var()
    dataframe[f'Ptp{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.ptp(x))
    dataframe[f'Percentile_10_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.percentile(x, 10))
    dataframe[f'Percentile_60_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.percentile(x, 60))
    dataframe[f'Percentile_90_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.percentile(x, 90))
    dataframe[f'quantile_5_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.quantile(x, 0.05))
    dataframe[f'quantile_95_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.quantile(x, 0.95))
    dataframe[f'quantile_99_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.quantile(x, 0.99))

    return dataframe

In [19]:
#generate_stats_columns(market_data, '<VOL>', 60)
#generate_stats_columns(market_data, '<CLOSE>', 60)
#generate_stats_columns(market_data, 'RSI', 60)
#generate_stats_columns(market_data, 'MACD', 60)
generate_stats_columns(market_data, '<VOL>', 30)
generate_stats_columns(market_data, '<CLOSE>', 30)
generate_stats_columns(market_data, 'RSI', 30)
generate_stats_columns(market_data, 'MACD', 30)
generate_stats_columns(market_data, '<VOL>', 120)
generate_stats_columns(market_data, '<CLOSE>', 120)
generate_stats_columns(market_data, 'RSI', 120)
generate_stats_columns(market_data, 'MACD', 120)

C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\1784763899.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe[f'Percentile_10_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.percentile(x, 10))
C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\1784763899.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataframe[f'Percentile_60_{column}_{window}'] = dataframe[column].rolling(window).apply(lambda x : np.percentile(x, 60))
C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\1784763899.py:10

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<VOL>,<SPREAD>,<HOUR>,...,MinMACD_120,StdMACD_120,VarMACD_120,PtpMACD_120,Percentile_10_MACD_120,Percentile_60_MACD_120,Percentile_90_MACD_120,quantile_5_MACD_120,quantile_95_MACD_120,quantile_99_MACD_120
0,2020.10.15,12:22:00,98870,98905,98840,98895,4570,24840,5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.10.15,12:23:00,98900,98935,98880,98900,5152,19660,5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020.10.15,12:24:00,98900,98945,98890,98935,4041,16365,5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020.10.15,12:25:00,98940,99010,98920,98950,10655,38331,5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.10.15,12:26:00,98955,98980,98930,98945,3412,12524,5,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282725,2022.12.06,17:56:00,110540,110545,110450,110495,7707,27987,5,17,...,-55.924562,55.775361,3110.890878,236.025161,-21.75011,15.730321,106.681122,-35.151687,171.625573,178.469279
282726,2022.12.06,17:57:00,110500,110520,110430,110460,4475,16087,5,17,...,-55.924562,57.083802,3258.560475,236.025161,-21.75011,16.458796,127.761020,-35.151687,171.625573,178.469279
282727,2022.12.06,17:58:00,110465,110510,110390,110450,7481,25731,5,17,...,-55.924562,58.135397,3379.724441,236.025161,-21.75011,16.818025,142.244368,-35.151687,171.625573,178.469279
282728,2022.12.06,17:59:00,110455,110500,110425,110465,4074,10868,5,17,...,-55.924562,58.998570,3480.831283,236.025161,-21.75011,17.183644,142.244368,-35.151687,171.625573,178.469279


Criando a veriável Target,
<br> sendo 1 tendencia de alta, 0 lateral e -1 baixa

In [20]:
def wma_target_trend(df, period):
    for i in range(period, len(df), period):
        temp_df = df.iloc[i - period: i]
        temp_df = list(temp_df)
        normalized_wma = (((temp_df[-1] - min(temp_df)) / (max(temp_df) - min(temp_df)) * 2 ) - 1)
        if normalized_wma > 0.5:
            df.iloc[i - period: i] = 1
        elif normalized_wma < -0.5:
            df.iloc[i - period: i] = -1
        else:
            df.iloc[i - period: i] = 0
            
    temp_df = df.iloc[i:]
    temp_df = list(temp_df)
    normalized_wma = (((temp_df[-1] - min(temp_df)) / (max(temp_df) - min(temp_df)) * 2 ) - 1)
    if normalized_wma > 0.5:
        df.iloc[i:] = 1
    elif normalized_wma < -0.5:
        df.iloc[i:] = -1
    else:
        df.iloc[i:] = 0
       
    return df

In [21]:
market_data = market_data.dropna()
market_data = market_data.reset_index(drop=True)

In [22]:
market_data['Trend'] = wma_target_trend(market_data['WMA'], 100)

C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\2684385078.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i - period: i] = 1
C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\2684385078.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i - period: i] = -1
C:\Users\Felps\AppData\Local\Temp\ipykernel_17496\2684385078.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i - period: i] = 0
C:\Users\Felps\Ap

In [23]:
market_data.to_csv('./WINM1_PROCESSED.CSV')

In [24]:
pd.DataFrame(market_data.columns).to_csv('./colunas.csv')

In [25]:
colunas = pd.read_csv('./colunas.csv')
len(colunas.columns)

2

In [26]:
colunas_selecionadas = []
for i in range(len(colunas)):
  colunas_selecionadas.append(colunas.iloc[i][1])
  print(colunas.iloc[i][1])

<DATE>
<TIME>
<OPEN>
<HIGH>
<LOW>
<CLOSE>
<TICKVOL>
<VOL>
<SPREAD>
<HOUR>
<MINUTE>
<DAY>
<YEAR>
<WEEKDAY>
<MONTH>
Close
RSI
RSI_Buy
RSI_Sell
SMA
SMA_Buy
SMA_Sell
ROC
ROC_Buy
ROC_Sell
EMA
EMA_Buy
EMA_Sell
WMA
WMA_Buy
WMA_Sell
MACD
MACD_Buy
MACD_Sell
<CLOSE>X<VOL>
<CLOSE>2
<VOL>2
<CLOSE>XRSI
<CLOSE>2
RSI2
<CLOSE>XSMA
<CLOSE>2
SMA2
<CLOSE>XROC
<CLOSE>2
ROC2
<CLOSE>XEMA
<CLOSE>2
EMA2
<CLOSE>XWMA
<CLOSE>2
WMA2
<CLOSE>XMACD
<CLOSE>2
MACD2
RSI_Buy_SMA_Buy
RSI_Buy_ROC_Buy
RSI_Buy_EMA_Buy
RSI_Buy_WMA_Buy
RSI_Buy_MACD_Buy
SMA_Buy_ROC_Buy
SMA_Buy_EMA_Buy
SMA_Buy_WMA_Buy
SMA_Buy_MACD_Buy
ROC_Buy_EMA_Buy
ROC_Buy_WMA_Buy
ROC_Buy_MACD_Buy
EMA_Buy_WMA_Buy
EMA_Buy_MACD_Buy
WMA_Buy_MACD_Buy
RSI_Sell_SMA_Sell
RSI_Sell_ROC_Sell
RSI_Sell_EMA_Sell
RSI_Sell_WMA_Sell
RSI_Sell_MACD_Sell
SMA_Sell_ROC_Sell
SMA_Sell_EMA_Sell
SMA_Sell_WMA_Sell
SMA_Sell_MACD_Sell
ROC_Sell_EMA_Sell
ROC_Sell_WMA_Sell
ROC_Sell_MACD_Sell
EMA_Sell_WMA_Sell
EMA_Sell_MACD_Sell
WMA_Sell_MACD_Sell
Max<VOL>_30
Mean<VOL>_30
Min<VOL>_30
Std<VO

In [27]:
len(colunas_selecionadas)

182

In [28]:
market_data.info

<bound method DataFrame.info of             <DATE>    <TIME>  <OPEN>  <HIGH>   <LOW>  <CLOSE>  <TICKVOL>  \
0       2020.10.15  14:21:00   99020   99050   99005    99030       4412   
1       2020.10.15  14:22:00   99035   99050   98995    99040       6127   
2       2020.10.15  14:23:00   99045   99055   99000    99040       4492   
3       2020.10.15  14:24:00   99035   99065   98985    99030       8250   
4       2020.10.15  14:25:00   99035   99060   99030    99045       4028   
...            ...       ...     ...     ...     ...      ...        ...   
282606  2022.12.06  17:56:00  110540  110545  110450   110495       7707   
282607  2022.12.06  17:57:00  110500  110520  110430   110460       4475   
282608  2022.12.06  17:58:00  110465  110510  110390   110450       7481   
282609  2022.12.06  17:59:00  110455  110500  110425   110465       4074   
282610  2022.12.06  18:00:00  110465  110475  110365   110405       5455   

        <VOL>  <SPREAD>  <HOUR>  ...  StdMACD_120  VarM

In [29]:
market_data[[
    '<CLOSE>'
,'<VOL>'
,'RSI_Buy'
,'RSI_Sell'
,'SMA_Buy'
,'SMA_Sell'
,'ROC_Buy'
,'ROC_Sell'
,'EMA_Buy'
,'EMA_Sell'
,'WMA_Buy'
,'WMA_Sell'
,'MACD'
,'MACD_Buy'
,'MACD_Sell'
,'<CLOSE>X<VOL>'
,'<CLOSE>2'
,'<VOL>2'
,'<CLOSE>XMACD'
,'<CLOSE>2'
,'MACD2'
,'RSI_Buy_SMA_Buy'
,'RSI_Buy_ROC_Buy'
,'RSI_Buy_EMA_Buy'
,'RSI_Buy_WMA_Buy'
,'RSI_Buy_MACD_Buy'
,'SMA_Buy_ROC_Buy'
,'SMA_Buy_EMA_Buy'
,'SMA_Buy_WMA_Buy'
,'SMA_Buy_MACD_Buy'
,'ROC_Buy_EMA_Buy'
,'ROC_Buy_WMA_Buy'
,'ROC_Buy_MACD_Buy'
,'EMA_Buy_WMA_Buy'
,'EMA_Buy_MACD_Buy'
,'WMA_Buy_MACD_Buy'
,'RSI_Sell_SMA_Sell'
,'RSI_Sell_ROC_Sell'
,'RSI_Sell_EMA_Sell'
,'RSI_Sell_WMA_Sell'
,'RSI_Sell_MACD_Sell'
,'SMA_Sell_ROC_Sell'
,'SMA_Sell_EMA_Sell'
,'SMA_Sell_WMA_Sell'
,'SMA_Sell_MACD_Sell'
,'ROC_Sell_EMA_Sell'
,'ROC_Sell_WMA_Sell'
,'ROC_Sell_MACD_Sell'
,'EMA_Sell_WMA_Sell'
,'EMA_Sell_MACD_Sell'
,'WMA_Sell_MACD_Sell'
,'<HOUR_SIN>'
,'<HOUR_COSINE>'
,'<MINUTE_SIN>'
,'<MINUTE_COSINE>'
,'<DAY_SIN>'
,'<DAY_COSINE>'
,'<YEAR_SIN>'
,'<YEAR_COSINE>'
,'<WEEKDAY_SIN>'
,'<WEEKDAY_COSINE>'
,'<MONTH_SIN>'
,'<MONTH_COSINE>'
,'Max<VOL>'
,'Mean<VOL>'
,'Min<VOL>'
,'Std<VOL>'
,'Var<VOL>'
,'Ptp<VOL>'
,'Percentile_10_<VOL>'
,'Percentile_60_<VOL>'
,'Percentile_90_<VOL>'
,'quantile_5_<VOL>'
,'quantile_95_<VOL>'
,'quantile_99_<VOL>'
,'Max<CLOSE>'
,'Mean<CLOSE>'
,'Min<CLOSE>'
,'Std<CLOSE>'
,'Var<CLOSE>'
,'Ptp<CLOSE>'
,'Percentile_10_<CLOSE>'
,'Percentile_60_<CLOSE>'
,'Percentile_90_<CLOSE>'
,'quantile_5_<CLOSE>'
,'quantile_95_<CLOSE>'
,'quantile_99_<CLOSE>'
,'MaxMACD'
,'MeanMACD'
,'MinMACD'
,'StdMACD'
,'VarMACD'
,'PtpMACD'
,'Percentile_10_MACD'
,'Percentile_60_MACD'
,'Percentile_90_MACD'
,'quantile_5_MACD'
,'quantile_95_MACD'
,'quantile_99_MACD'
,'Trend'
]].corr()

KeyError: "['<HOUR_SIN>', '<HOUR_COSINE>', '<MINUTE_SIN>', '<MINUTE_COSINE>', '<DAY_SIN>', '<DAY_COSINE>', '<YEAR_SIN>', '<YEAR_COSINE>', '<WEEKDAY_SIN>', '<WEEKDAY_COSINE>', '<MONTH_SIN>', '<MONTH_COSINE>', 'Max<VOL>', 'Mean<VOL>', 'Min<VOL>', 'Std<VOL>', 'Var<VOL>', 'Ptp<VOL>', 'Percentile_10_<VOL>', 'Percentile_60_<VOL>', 'Percentile_90_<VOL>', 'quantile_5_<VOL>', 'quantile_95_<VOL>', 'quantile_99_<VOL>', 'Max<CLOSE>', 'Mean<CLOSE>', 'Min<CLOSE>', 'Std<CLOSE>', 'Var<CLOSE>', 'Ptp<CLOSE>', 'Percentile_10_<CLOSE>', 'Percentile_60_<CLOSE>', 'Percentile_90_<CLOSE>', 'quantile_5_<CLOSE>', 'quantile_95_<CLOSE>', 'quantile_99_<CLOSE>', 'MaxMACD', 'MeanMACD', 'MinMACD', 'StdMACD', 'VarMACD', 'PtpMACD', 'Percentile_10_MACD', 'Percentile_60_MACD', 'Percentile_90_MACD', 'quantile_5_MACD', 'quantile_95_MACD', 'quantile_99_MACD'] not in index"